In [56]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Optional: also set width to avoid line wrapping
pd.set_option('display.width', None)

# Optional: disable column width truncation
pd.set_option('display.max_colwidth', None)

In [ ]:
all_players = nfl.import_players()
all_qbs = all_players[all_players['position'] == 'QB']
all_qbs.reset_index(inplace=True, drop='index')
all_qbs_id = list(all_qbs.gsis_id)

2024 done.
Downcasting floats.


id
00-0023459     3.0
00-0026158     1.0
00-0026300     0.0
00-0026498     2.0
00-0027973     1.0
              ... 
00-0039917     0.0
00-0039918    10.0
00-0039919     0.0
00-0039920     0.0
00-0039921     0.0
Name: two_point_attempt, Length: 367, dtype: float32

In [ ]:
weekly_data = nfl.import_weekly_data(years=[2024])
weekly_data['completion_pct'] = weekly_data.completions / weekly_data.attempts
weekly_relevant = weekly_data[weekly_data['player_id'].isin(all_qbs_id)][['player_id', 'passing_tds', 'interceptions', 
                                                                          'sacks', 'sack_fumbles', 'rushing_tds', 
                                                                          'rushing_fumbles', 'completion_pct', 'passing_air_yards', 'attempts']]
weekly_relevant = weekly_relevant.groupby('player_id').agg({
    'passing_tds' : 'sum', 
    'interceptions' : 'sum',
    'sacks' : 'sum',
    'sack_fumbles' : 'sum',
    'rushing_tds' : 'sum',
    'rushing_fumbles' : 'sum', 
    'completion_pct' : 'mean',
    'passing_air_yards' : 'sum',
    'attempts' : 'sum'})

Downcasting floats.


In [116]:
stats2024 = nfl.import_pbp_data(years=[2024])
stats2024 = stats2024[(stats2024['qb_kneel'] == 0.0) & (stats2024['play_type'] != 'no_play')]
relevant = stats2024[['play_type', 'shotgun', 'qb_dropback', 'qb_scramble', 'air_yards', 'rushing_yards', 'id']]
relevant = relevant[relevant['id'].isin(all_qbs_id)]

2024 done.
Downcasting floats.


In [117]:
pass_stats = relevant[relevant['play_type'] == 'pass'].groupby('id').agg({'shotgun' : sum, 'qb_dropback' : sum})
pass_stats['shotgun'] = pass_stats.shotgun / pass_stats.qb_dropback
pass_stats.columns = ['shotgun_rate', 'total_passes']
pass_stats

,shotgun_rate,total_passes
id,,
00-0023459,0.842105,627.0
00-0026158,0.932584,267.0
00-0026300,0.500000,4.0
00-0026498,0.706731,624.0
00-0027973,0.797619,168.0
...,...,...
00-0039732,0.856911,615.0
00-0039851,0.882353,374.0
00-0039910,0.962963,648.0


In [119]:
run_stats = relevant[relevant['play_type'] == 'run'].groupby('id').agg({'qb_scramble' : ['sum', 'count'], 'rushing_yards' : 'sum'})
run_stats.columns = ['qb_scramble_sum', 'total_runs', 'rushing_yards']
run_stats['qb_scramble_rate'] = run_stats.qb_scramble_sum / run_stats.total_runs
run_stats = run_stats[['qb_scramble_rate', 'total_runs', 'rushing_yards']]
run_stats

,qb_scramble_rate,total_runs,rushing_yards
id,,,
00-0023459,0.941176,17,111.0
00-0026158,0.500000,6,29.0
00-0026300,0.000000,2,3.0
00-0026498,0.235294,17,64.0
00-0027973,0.857143,7,38.0
...,...,...,...
00-0039732,0.654321,81,491.0
00-0039851,0.865385,52,423.0
00-0039910,0.508876,169,1044.0


In [ ]:
total_data = weekly_relevant.join(pass_stats).join(run_stats)

total_data['sacks'] = total_data['sacks'] / total_data['total_passes']
total_data['sack_fumbles'] = total_data['sack_fumbles'] / total_data['total_passes']

total_data['interceptions'] = total_data['interceptions'] / total_data['attempts']
total_data['passing_tds'] = total_data['passing_tds'] / total_data['attempts']
total_data['passing_air_yards'] = total_data['passing_air_yards'] / total_data['attempts']

total_data['rushing_tds'] = total_data['rushing_tds'] / total_data['total_runs']
total_data['rushing_fumbles'] = total_data['rushing_fumbles'] / total_data['total_runs']
total_data['rushing_yards'] = total_data['rushing_yards'] / total_data['total_runs']

total_data['pass_pct'] = total_data.total_passes / (total_data.total_passes + total_data.total_runs)

total_data = total_data[['sacks', 'sack_fumbles', 'interceptions', 
                         'passing_tds', 'passing_air_yards', 'rushing_tds', 
                         'rushing_fumbles', 'rushing_yards', 'pass_pct',
                         'shotgun_rate', 'qb_scramble_rate', 'completion_pct']]

In [125]:
total_data

,passing_tds,interceptions,sacks,sack_fumbles,rushing_tds,rushing_fumbles,completion_pct,passing_air_yards,attempts,shotgun_rate,total_passes,qb_scramble_rate,total_runs,rushing_yards,pass_pct
player_id,,,,,,,,,,,,,,,
00-0023459,0.047945,0.018836,0.063796,0.007974,0.000000,0.000000,0.634684,6.873288,584,0.842105,627.0,0.941176,17.0,6.529412,0.973602
00-0026158,0.048387,0.028226,0.067416,0.011236,0.000000,0.166667,0.648427,8.931452,248,0.932584,267.0,0.500000,6.0,4.833333,0.978022
00-0026300,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.750000,17.666667,3,0.500000,4.0,0.000000,2.0,1.500000,0.666667
00-0026498,0.040816,0.013605,0.056090,0.008013,0.000000,0.176471,0.656563,7.671769,588,0.706731,624.0,0.235294,17.0,3.764706,0.973479
00-0027973,0.043750,0.037500,0.041667,0.005952,0.000000,0.000000,0.557045,6.637500,160,0.797619,168.0,0.857143,7.0,5.428571,0.960000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00-0039732,0.050934,0.020374,0.042276,0.003252,0.049383,0.012346,0.659797,7.429542,589,0.856911,615.0,0.654321,81.0,6.061728,0.883621
00-0039851,0.044379,0.029586,0.090909,0.013369,0.038462,0.076923,0.595147,7.316568,338,0.882353,374.0,0.865385,52.0,8.134615,0.877934
00-0039910,0.050505,0.016835,0.078704,0.003086,0.041420,0.017751,0.701405,7.196970,594,0.962963,648.0,0.508876,169.0,6.177515,0.793146
